In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import re

In [2]:
datas = pd.read_csv('./timetables.csv', sep='\t')
print(datas.head())

                 trip_id                                             trajet  \
0  OCESN003100F140147152         Gare de Le Havre - Gare de Paris-St-Lazare   
1  OCESN003190F040047309           Gare de Dieppe - Gare de Paris-St-Lazare   
2  OCESN003198F030037315  Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...   
3  OCESN003300F030037323        Gare de Cherbourg - Gare de Paris-St-Lazare   
4  OCESN003313F380387526             Gare de Caen - Gare de Paris-St-Lazare   

   duree  
0    138  
1    145  
2     97  
3    194  
4    149  


In [3]:
s = pd.Series(datas['trajet'])
series = s.str.split(r" - ", expand=True)
print(series[0], series[1])

0              Gare de Le Havre
1                Gare de Dieppe
2       Gare de Paris-St-Lazare
3             Gare de Cherbourg
4                  Gare de Caen
                 ...           
1570      Gare de Belfort-Ville
1571    Gare de Lons-le-Saunier
1572      Gare de Belfort-Ville
1573    Gare de Besançon-Viotte
1574    Gare de Bourg-en-Bresse
Name: 0, Length: 1575, dtype: object 0         Gare de Paris-St-Lazare
1         Gare de Paris-St-Lazare
2       Gare de Rouen-Rive-Droite
3         Gare de Paris-St-Lazare
4         Gare de Paris-St-Lazare
                  ...            
1570        Gare de Lyon-Perrache
1571        Gare de Lyon-Perrache
1572      Gare de Lons-le-Saunier
1573      Gare de Lons-le-Saunier
1574             Gare de Mouchard
Name: 1, Length: 1575, dtype: object


In [4]:
departures = series[0]
destinations = series[1]
datas = datas.join(departures)
datas = datas.join(destinations)

In [5]:
datas = datas.rename(columns={0: "departures", 1: "destinations", "duree":"weight"})
datas

,trip_id,trajet,weight,departures,destinations
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138,Gare de Le Havre,Gare de Paris-St-Lazare
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145,Gare de Dieppe,Gare de Paris-St-Lazare
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97,Gare de Paris-St-Lazare,Gare de Rouen-Rive-Droite
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194,Gare de Cherbourg,Gare de Paris-St-Lazare
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149,Gare de Caen,Gare de Paris-St-Lazare
...,...,...,...,...,...
1570,OCESN895822F0500552575,Gare de Belfort-Ville - Gare de Lyon-Perrache,244,Gare de Belfort-Ville,Gare de Lyon-Perrache
1571,OCESN895830F0200252600,Gare de Lons-le-Saunier - Gare de Lyon-Perrache,103,Gare de Lons-le-Saunier,Gare de Lyon-Perrache
1572,OCESN895880F0500552634,Gare de Belfort-Ville - Gare de Lons-le-Saunier,144,Gare de Belfort-Ville,Gare de Lons-le-Saunier
1573,OCESN895940F0200252654,Gare de Besançon-Viotte - Gare de Lons-le-Saunier,89,Gare de Besançon-Viotte,Gare de Lons-le-Saunier


In [6]:
G_gares = nx.from_pandas_edgelist(datas, source="departures", target="destinations", edge_attr='weight')
G_gares.edges()

EdgeView([('Gare de Le Havre', 'Gare de Paris-St-Lazare'), ('Gare de Le Havre', 'Gare de Montivilliers'), ('Gare de Le Havre', 'Gare de Rolleville'), ('Gare de Le Havre', 'Gare de Fécamp'), ('Gare de Le Havre', 'Gare de Rouen-Rive-Droite'), ('Gare de Paris-St-Lazare', 'Gare de Dieppe'), ('Gare de Paris-St-Lazare', 'Gare de Rouen-Rive-Droite'), ('Gare de Paris-St-Lazare', 'Gare de Cherbourg'), ('Gare de Paris-St-Lazare', 'Gare de Caen'), ('Gare de Paris-St-Lazare', 'Gare de Trouville-Deauville'), ('Gare de Paris-St-Lazare', 'Gare de Vernon-Giverny'), ('Gare de Paris-St-Lazare', 'Gare de Oissel'), ('Gare de Paris-St-Lazare', 'Gare de Serquigny'), ('Gare de Dieppe', 'Gare de Rouen-Rive-Droite'), ('Gare de Rouen-Rive-Droite', 'Gare de Caen'), ('Gare de Rouen-Rive-Droite', 'Gare de Mantes-la-Jolie'), ('Gare de Rouen-Rive-Droite', 'Gare de Amiens'), ('Gare de Rouen-Rive-Droite', 'Gare de Lille Flandres'), ('Gare de Rouen-Rive-Droite', 'Gare de Yvetot'), ('Gare de Cherbourg', 'Gare de Caen'),

In [7]:
court_chemin_array = nx.shortest_path(G_gares)
Lille_Paris = court_chemin_array["Gare de Paris-St-Lazare"]["Gare de Lille Flandres"]
print(Lille_Paris)
time_list = []
for index, elem in enumerate(Lille_Paris):
    if index+1 == len(Lille_Paris):
        break;
#         prev_el = str(Lille[index-1])
    curr_el = elem
    next_el = Lille_Paris[index+1]
    
    time = nx.path_weight(G_gares, [curr_el, next_el], weight="weight")
    time_list.append(time)
    print(time)
    

print(sum(time_list))  

['Gare de Paris-St-Lazare', 'Gare de Rouen-Rive-Droite', 'Gare de Lille Flandres']
97
168
265


In [8]:
nx.path_weight(G_gares, ['Gare de Lille Flandres', 'Gare de Paris-St-Lazare'], weight="weight")

NetworkXNoPath: path does not exist

In [10]:
END = '0e8ed89a-47ba-4cdb-938e-b8af8e084d5c'

# Text attributes
ALL_OFF = '\033[0m'
BOLD = '\033[1m'
FG_RED = '\033[31m'
class pretty_output():
    '''
    Context manager for pretty terminal prints
    '''

    def __init__(self, *attr):
        self.attributes = attr

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

    def write(self, msg):
        style = ''.join(self.attributes)
        print('{}{}{}'.format(style, msg.replace(END, ALL_OFF + style), ALL_OFF))

In [11]:
def checkDestination(dep, dest, first, second):
    dep = dep.capitalize()
    dest = dest.capitalize()
    for u in G_gares.nodes():
        first.append(u)
    for v in G_gares.nodes():
        second.append(v)
        
    allee = list(filter(lambda x: dep in x, first))
    
    arrivee = list(filter(lambda x: dest in x, second))

    return (allee, arrivee)
checkDestination("lille", "Paris", [], [])

(['Gare de Lille Flandres', 'Gare de Lille Europe'],
 ['Gare de Paris-St-Lazare',
  'Gare de Paris-Montp.3-Vaug.',
  'Gare de Paris-Montparnasse 1-2',
  'Gare de Paris-Austerlitz',
  'Gare de Paris-Bercy',
  'Gare de Paris-Gare-de-Lyon',
  'Gare de Paris Gare du Nord',
  'Gare de Paris-Est'])

In [55]:
def getAvailableShortPath(departure, destination):
    
    departure_list = []
    destination_list = []
    #vérification si le départ ou l'arrivée est bien un mot raisonnable en excluant des mots vides et ceux avec une longueur 
    # inférieure ou égale à 3
    
    if departure == "" or len(departure) <= 3:
        with pretty_output(BOLD, FG_RED) as out:
            out.write("Désolé Madame/Monsieur, on n'a pas saisi votre lieu de départ, Pouvez-vous réessayer ultérieurement?")
    
    elif destination == "" or len(destination) <= 3:
        with pretty_output(BOLD, FG_RED) as out:
            out.write("Désolé Madame/Monsieur, on n'a pas saisi votre lieu d'arrivée, Pouvez-vous réessayer ultérieurement?")
    # verifier si le mot inséré est contenue dans l'une des gares qu'on a dans le dataset
    
    else:
        # récupérer tous les gares correspondant au mot inséré
        depart, arrivee = checkDestination(departure, destination, departure_list, destination_list)

        if len(depart)==0 or len(arrivee)==0:
            with pretty_output(BOLD, FG_RED) as out:
                out.write("Désolé Madame/Monsieur, nous n'avons aucun trajet correspondant à votre demande!")
        
        for k in depart:
            for l in arrivee:
                couple = (k,l)
                short_path = nx.shortest_path(G_gares)
                court = short_path[couple[0]][couple[1]]
                return court
#                         with pretty_output(BOLD, FG_RED) as out:
#                     out.write(f"Bonjour Madame/Monsieur, pour aller à {destination.capitalize()}, le plus court chemin d'apres nos trajets est {court}")
            


getAvailableShortPath("lille", "paris")

['Gare de Lille Flandres',
 'Gare de Rouen-Rive-Droite',
 'Gare de Paris-St-Lazare']